In [1]:
import os
import gc
import numpy as np
import mne
from mne.preprocessing import ICA, create_eog_epochs

In [2]:
CANONICAL_CHANNELS = [
    "EEG Fp1", "EEG F3", "EEG C3", "EEG P3", "EEG O1",
    "EEG F7", "EEG T3", "EEG T5",
    "EEG Fc1", "EEG Fc5",
    "EEG Cp1", "EEG Cp5",
    "EEG F9", "EEG Fz", "EEG Pz",
    "EEG F4", "EEG C4", "EEG P4", "EEG O2",
    "EEG F8", "EEG T4", "EEG T6",
    "EEG Fc2", "EEG Fc6",
    "EEG Cp2", "EEG Cp6",
    "EEG F10"
]


In [3]:
#change paths
RAW_PATH = "/workspace/raw_dataset"
SAVE_PATH = "/workspace/preprocessed_data_2"
os.makedirs(SAVE_PATH, exist_ok=True)

In [4]:
# Parameters

# Bandpass filtering
LOW_F = 0.5
HIGH_F = 40

# Notch filtering
NOTCH = 50

# Epoching
EPOCH_LEN = 5.0

# Resampling
RESAMPLE_SFREQ = 256

# Memory safety: limit per-file duration (IMPORTANT)
MAX_DURATION = 600    # seconds = 10 minutes

# ICA (currently disabled in pipeline)
ICA_COMPONENTS = 20
ICA_MAX_DURATION = 20 * 60   # first 20 minutes only
ICA_RANDOM_STATE = 97


In [5]:
for root, _, files in os.walk(RAW_PATH):
    for file in files:
        if not file.endswith(".edf"):
            continue

        edf_path = os.path.join(root, file)
        save_name = file.replace(".edf", ".npz")
        save_path = os.path.join(SAVE_PATH, save_name)

        if os.path.exists(save_path):
            print("Skipping (already done):", file)
            continue

        print("\nProcessing:", file)

        try:
            # --------------------------------------------------
            # 1. Load EDF safely (no preload)
            # --------------------------------------------------
            raw = mne.io.read_raw_edf(
                edf_path,
                preload=False,
                verbose=False
            )

            if raw.n_times < 1:
                print("Empty file, skipping:", file)
                del raw
                continue

            raw.pick("eeg")

            # --------------------------------------------------
            # 2. Global duration cap (RAM protection)
            # --------------------------------------------------
            raw.crop(tmax=min(raw.times[-1], MAX_DURATION))
            raw.load_data()

            # --------------------------------------------------
            # 3. Filtering
            # --------------------------------------------------
            raw.filter(LOW_F, HIGH_F, fir_design="firwin", verbose=False)
            raw.notch_filter(NOTCH, verbose=False)

            # --------------------------------------------------
            # 4. Average reference
            # --------------------------------------------------
            raw.set_eeg_reference("average", verbose=False)

            # --------------------------------------------------
            # 5. Resample (major memory win)
            # --------------------------------------------------
            raw.resample(RESAMPLE_SFREQ, verbose=False)

            # --------------------------------------------------
            # 6. ICA (fit on short data only)
            # --------------------------------------------------
            raw_ica = raw.copy().crop(
                tmax=min(raw.times[-1], ICA_MAX_DURATION)
            )

            ica = ICA(
                n_components=ICA_COMPONENTS,
                random_state=ICA_RANDOM_STATE,
                method="fastica",
                max_iter="auto"
            )

            ica.fit(raw_ica)

            eog_inds = []
            try:
                eog_epochs = create_eog_epochs(raw_ica, verbose=False)
                eog_inds, _ = ica.find_bads_eog(eog_epochs)
            except Exception:
                pass

            muscle_inds, _ = ica.find_bads_muscle(raw_ica, threshold=0.7)

            ica.exclude = list(set(eog_inds + muscle_inds))
            print("Removing ICA components:", ica.exclude)

            raw = ica.apply(raw)

            del raw_ica, ica
            gc.collect()

            # --------------------------------------------------
            # 7. ENFORCE CANONICAL CHANNELS (ZERO-FILL)
            # --------------------------------------------------
            sfreq = raw.info["sfreq"]
            n_times = raw.n_times

            existing = {
                ch: raw.get_data(picks=ch)[0]
                for ch in raw.ch_names
            }

            data = np.zeros(
                (len(CANONICAL_CHANNELS), n_times),
                dtype=np.float32
            )

            for i, ch in enumerate(CANONICAL_CHANNELS):
                if ch in existing:
                    data[i] = existing[ch]

            del raw
            gc.collect()

            info = mne.create_info(
                ch_names=CANONICAL_CHANNELS,
                sfreq=sfreq,
                ch_types="eeg"
            )

            raw_fixed = mne.io.RawArray(data, info, verbose=False)

            # --------------------------------------------------
            # 8. Epoching
            # --------------------------------------------------
            epochs = mne.make_fixed_length_epochs(
                raw_fixed,
                duration=EPOCH_LEN,
                preload=True,
                verbose=False
            )

            if len(epochs) == 0:
                print("No epochs created:", file)
                del raw_fixed, epochs
                gc.collect()
                continue

            # --------------------------------------------------
            # 9. Extract data
            # --------------------------------------------------
            X = epochs.get_data().astype(np.float32)

            # --------------------------------------------------
            # 10. Normalize per epoch
            # --------------------------------------------------
            mean = X.mean(axis=-1, keepdims=True)
            std = X.std(axis=-1, keepdims=True) + 1e-6
            X = (X - mean) / std

            # --------------------------------------------------
            # 11. Labels (placeholder)
            # --------------------------------------------------
            y = np.zeros(len(X), dtype=np.int8)

            # --------------------------------------------------
            # 12. Save
            # --------------------------------------------------
            np.savez_compressed(save_path, X=X, y=y)
            print("Saved:", save_path)

            # --------------------------------------------------
            # 13. Cleanup
            # --------------------------------------------------
            del raw_fixed, epochs, X, y, mean, std
            gc.collect()

        except Exception as e:
            print("Error processing", file, ":", e)
            gc.collect()
            continue



Processing: PN00-1.edf
Reading 0 ... 307200  =      0.000 ...   600.000 secs...


/tmp/ipykernel_109/1729231461.py:20: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(
/tmp/ipykernel_109/1729231461.py:20: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(
/tmp/ipykernel_109/1729231461.py:20: RuntimeWarning: Highpass cutoff frequency 15.91549 is greater than lowpass cutoff frequency 15.0, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(


Fitting ICA to data using 35 channels (please be patient, this may take a while)
Selecting by number: 20 components
Fitting ICA took 6.7s.
Effective window size : 8.000 (s)
Removing ICA components: [5, 11, 13, 15, 16, 17]
Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 6 ICA components
    Projecting back using 35 PCA components


/tmp/ipykernel_109/1729231461.py:78: RuntimeWarning: No sensor positions found. Scores for bad muscle components are only based on the 'slope' criterion.
  muscle_inds, _ = ica.find_bads_muscle(raw_ica, threshold=0.7)


Saved: /workspace/preprocessed_data_2/PN00-1.npz

Processing: PN00-2.edf
Reading 0 ... 307200  =      0.000 ...   600.000 secs...


...

Fitting ICA to data using 49 channels (please be patient, this may take a while)
Selecting by number: 20 components
Fitting ICA took 103.8s.


/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:132: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Effective window size : 8.000 (s)
Removing ICA components: [10, 11]
Applying ICA to Raw instance
    Transforming to ICA space (20 components)
    Zeroing out 2 ICA components
    Projecting back using 49 PCA components


/tmp/ipykernel_109/1729231461.py:78: RuntimeWarning: No sensor positions found. Scores for bad muscle components are only based on the 'slope' criterion.
  muscle_inds, _ = ica.find_bads_muscle(raw_ica, threshold=0.7)


Saved: /workspace/preprocessed_data_2/PN17-2.npz


- saved as npz as its compressed stack of numpy arrays compatible with dl models

CNN
-  input layer
- 1 d convolution layer - kernel slides over the signal to find features
- activation layer - make the features non-linear
- batch normalisation - normalise feature maps - better generalisation
- pooling layer - samples temporal dimensions - robust to noise and change in temporal cahnges

- bilstm focuses on temporal features
- memory cells - 3 gates
- forget, input, cell state
- forget removes noise
- input - stores features
- cell state - accumalte all features
- forward - from past to present
- backward - present to past

- attention mechanism
- higher wts to all seizure features
